In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as f

import requests
from zipfile import ZipFile

In [2]:
json = requests.get('https://files.grouplens.org/datasets/movielens/ml-100k.zip', allow_redirects=True)
open('dataset.zip', 'wb').write(json.content) 

with ZipFile('dataset.zip', 'r') as zip:
    zip.printdir()
  
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

File Name                                             Modified             Size
ml-100k/                                       2016-01-29 14:26:28            0
ml-100k/allbut.pl                              2000-07-19 16:09:28          716
ml-100k/mku.sh                                 2000-07-19 16:09:28          643
ml-100k/README                                 2016-01-29 14:26:28         6750
ml-100k/u.data                                 2000-07-19 16:09:30      1979173
ml-100k/u.genre                                2000-07-19 16:09:30          202
ml-100k/u.info                                 2000-07-19 16:09:30           36
ml-100k/u.item                                 2000-07-19 16:09:30       236344
ml-100k/u.occupation                           2000-07-19 16:09:30          193
ml-100k/u.user                                 2000-07-19 16:09:30        22628
ml-100k/u1.base                                2001-03-08 12:33:08      1586544
ml-100k/u1.test                         

In [3]:
class RatingMatrix:

  def __init__(self, train=True):

    self.__ext = '.base'
    if not train:
      self.__ext = '.test'
    l = [i for i in range(1, 6)]
    self.__data = []
    self.__labels = []
    self.__next = 0

    for i in l:
      with open('ml-100k/u'+str(i)+self.__ext) as o:
        while True:
          line = o.readline()
          if not line:
            break
          toks = line.split('\t')
          self.__data.append([int(toks[0])-1, int(toks[1])-1])
          self.__labels.append(float(toks[2]))

  def __iter__(self):
    self.__next = 0
    return self

  def __next__(self):
    if self.__next == len(self.__data):
      self.__next = 0
      raise StopIteration
    val = (self.__data[self.__next], self.__labels[self.__next])
    self.__next += 1
    return val

class LatentFactors:

  def __init__(self, users, items):
    self.__users = users
    self.__items = items
    self.__user_lf = torch.rand(users, 100)
    self.__item_lf = torch.rand(items, 100)

  def get(self, user, item):
    third = torch.cat((self.__user_lf[user], self.__item_lf[item]), 0)
    return third
  
  def backward(self, user, item, gradient):
    self.__user_lf[user] -= 0.01*gradient[0]
    self.__item_lf[item] -= 0.01*gradient[1]

train_data = RatingMatrix(train=True)
lf = LatentFactors(943, 1682)

In [4]:
class LimiterFunction(torch.autograd.Function):

  def forward(ctx, input):
    ctx.save_for_backward(input)
    return input.clamp(min=0, max=5)

  def backward(ctx, grad_output):
    input, = ctx.saved_tensors
    grad_input = grad_output.clone()
    grad_input[input < 0] = 0
    grad_input[input > 5] = 0
    return grad_input

class DeepWideNet(nn.Module):

  def __init__(self):
    super(DeepWideNet, self).__init__()
    
    self.l1 = nn.Linear(200, 100)
    self.l2 = nn.Linear(100, 50)
    self.l3 = nn.Linear(50, 25)
    self.l4 = nn.Linear(25, 1)

    self.limiter = LimiterFunction.apply

  def forward(self, x):

    x = f.relu(self.l1(x))
    x = f.relu(self.l2(x))
    x = f.relu(self.l3(x))
    x = self.limiter(self.l4(x))
    return x

device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda:0")

net = DeepWideNet()
net.to(device)

DeepWideNet(
  (l1): Linear(in_features=200, out_features=100, bias=True)
  (l2): Linear(in_features=100, out_features=50, bias=True)
  (l3): Linear(in_features=50, out_features=25, bias=True)
  (l4): Linear(in_features=25, out_features=1, bias=True)
)

In [7]:
loss = nn.MSELoss()
opt = torch.optim.SGD(net.parameters(), lr=0.01)
epochs = 10
count = 0
running_loss = 0

for epoch in range(epochs):
  count = 0

  for (X, Y) in train_data:
    #print(X, Y)
    x = lf.get(X[0], X[1])
    x = x.to(device)
    opt.zero_grad()
    
    pred = net(x)
    act = torch.tensor([Y]).to(device)
    out = loss(pred, act)
    running_loss += out.item()
    out.backward()
    #p_l = list(net.parameters())
    #gr = torch.matmul(p_l[0].T, p_l[1].grad)
    #lf.backward(X[0], X[1], gr.view(-1, 100))
    opt.step()
    count += 1
  #print(running_loss/count)

In [8]:
print(running_loss/count)

1.1623523795302302
